In [5]:
import os
import pandas as pd
import glob

In [33]:
path = "../Data collection/Historic data/raw data/"
files = glob.glob(path + "/*.csv") 
Datasets = pd.DataFrame()
content = [pd.read_csv(filename, index_col=None) for filename in files]
Datasets = pd.concat(content)

Datasets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175954 entries, 0 to 4859
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   titre    175954 non-null  object
 1   date     175954 non-null  object
 2   extrait  175723 non-null  object
dtypes: object(3)
memory usage: 5.4+ MB


In [34]:
import dateparser

In [35]:
def format_date(date):
    return  dateparser.parse(str(date)).date()
Datasets["date"] = Datasets["date"].apply(format_date)

c:\Users\hp\PycharmProjects\pythonProject1\venv\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
